In [ ]:
!pip install mplfinance

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mplfinance as mpf
from statsmodels.stats.outliers_influence import variance_inflation_factor
import math
from google.colab import files


In [ ]:
df_first = pd.read_csv("/content/df.csv")



In [ ]:
df_first['Close'].plot()

In [ ]:
df = np.log(df_first['Close'])
df.plot()

In [ ]:
msk = (df.index < len(df)-180)
df_train = df[msk].copy()
df_test = df[~msk].copy()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

acf_original = plot_acf(df_train)

pacf_original = plot_pacf(df_train)

In [ ]:
from statsmodels.tsa.stattools import adfuller
adf_test = adfuller(df_train)
print(f'p-value: {adf_test[1]}')

In [ ]:
df_train_diff = df_train.diff().dropna()
df_train_diff.plot()

In [ ]:
acf_diff = plot_acf(df_train_diff)

pacf_diff = plot_pacf(df_train_diff)

In [ ]:
adf_test = adfuller(df_train_diff)
print(f'p-value: {adf_test[1]}')

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
model = ARIMA(df_train, order=(4,2,5))
model_fit = model.fit()
print(model_fit.summary())

In [ ]:
import matplotlib.pyplot as plt
residuals = model_fit.resid[1:]
fig, ax = plt.subplots(1,2)
residuals.plot(title='Residuals', ax=ax[0])
residuals.plot(title='Density', kind='kde', ax=ax[1])
plt.show()

In [ ]:
acf_res = plot_acf(residuals)

pacf_res = plot_pacf(residuals)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import numpy as np

# Dữ liệu log toàn bộ
df_log = np.log(df_first['Close'])

# Fit ARIMA trên toàn bộ dữ liệu, dùng cùng order (4,2,5)
model_full = ARIMA(df_log, order=(4,2,5))
model_fit_full = model_full.fit()


forecast_next_20 = model_fit_full.forecast(steps=20)


print("Forecast (log scale):")
print(forecast_next_20)

# Chuyển về giá trị thực (Close)
forecast_next_20_actual = np.exp(forecast_next_20)
print("\nForecast (actual Close):")
print(forecast_next_20_actual)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Chuyển cột Date về datetime nếu chưa
df_first['Date'] = pd.to_datetime(df_first['Date'])

# Lọc dữ liệu từ 2025 trở đi
df_2025 = df_first[df_first['Date'].dt.year >= 2025]

# Dữ liệu thực tế
actual = df_2025['Close']

# Forecast từ ARIMA (giá trị thực)
forecast_next_20_actual = np.exp(model_fit_full.forecast(steps=30))

# Tạo index cho forecast dựa trên ngày tiếp theo sau dữ liệu 2025
last_date = df_2025['Date'].iloc[-1]
forecast_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=30, freq='B')  # B = ngày làm việc

# Vẽ biểu đồ
plt.figure(figsize=(14,6))
plt.plot(df_2025['Date'], actual, label='Actual Close', color='blue')
plt.plot(forecast_dates, forecast_next_20_actual, label='Forecast ARIMA 30 steps', color='red', linestyle='--')
plt.title('Close Price vs ARIMA 30-step Forecast from 2025')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.show()



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Forecast từ ARIMA (giá trị thực)
forecast_next_30_actual = np.exp(model_fit_full.forecast(steps=30))

# Tạo index cho forecast
forecast_index = np.arange(len(df_first), len(df_first) + len(forecast_next_30_actual))

# Vẽ biểu đồ chỉ phần dự đoán
plt.figure(figsize=(12,5))
plt.plot(forecast_index, forecast_next_30_actual, label='Forecast ARIMA 30 steps', color='red', linestyle='--', marker='o')
plt.title('Zoom: ARIMA 30-step Forecast')
plt.xlabel('Time (Index)')
plt.ylabel('Close Price')
plt.legend()
plt.show()
